L’analyse de sentiments est l’une des applications qu’on rencontre couramment en NLP. En effet, pour beaucoup de business, il est important de connaître l’avis des clients/utilisateurs. Un outil qui permet de déterminer si un produit ou service est plutôt bien vu ou plutôt mal vu, de façon automatique en se basant sur du texte écrit en langage naturel, est forcément une bonne idée. Dans cet article, je vous propose d’implémenter un système d’analyse de sentiments en utilisant notre model puis utilisation le modèle “cousin” de BERT CamemBERT.



### Creation model LTSM

In [ ]:
import torch
import seaborn
import pandas as pd
from sklearn import metrics
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip  /content/drive/MyDrive/ML/NLP/archive.zip

Archive:  /content/drive/MyDrive/ML/NLP/archive.zip
  inflating: test.csv                
  inflating: train.csv               
  inflating: valid.csv               


Le jeu de données est constitué de reviews de films ressemblant à ça :


In [ ]:
dataset = pd.read_csv("/content/train.csv")
dataset.head(5)

,Unnamed: 0,film-url,review,polarity
0,0,http://www.allocine.fr/film/fichefilm-135259/c...,Si vous cherchez du cinéma abrutissant à tous ...,0
1,1,http://www.allocine.fr/film/fichefilm-172430/c...,"Trash, re-trash et re-re-trash...! Une horreur...",0
2,2,http://www.allocine.fr/film/fichefilm-15105/cr...,"Et si, dans les 5 premières minutes du film, l...",0
3,3,http://www.allocine.fr/film/fichefilm-188629/c...,Mon dieu ! Quelle métaphore filée ! Je suis ab...,0
4,4,http://www.allocine.fr/film/fichefilm-23514/cr...,"Premier film de la saga Kozure Okami, ""Le Sabr...",1


In [ ]:
reviews = dataset['review'].values.tolist()
sentiments = dataset['polarity'].values.tolist()

Pour pouvoir passer nos données au modèle, il va falloir qu’elles soient encodées. Encoder, c’est donner une représentation numérique (en vrai vectorielle) au texte exactement comme le fait un embedding classique. La bibliothèque keras , tokenize() fournit un analyseur lexical  L'analyseur de ce module renvoie également des jetons en donne les commantaires a ce analyseur et par la methode texts_to_sequences() en Transforme chaque commantaire a une séquence d'entiers.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import OneHotEncoder

max_fatures = 512
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(reviews)
X = tokenizer.texts_to_sequences(reviews)
X = pad_sequences(X)

sentiments = torch.tensor(sentiments)

sentiments
y = pd.get_dummies(sentiments)

In [ ]:
print(reviews[0])
print(X[0])
print(y[0][0])

Si vous cherchez du cinéma abrutissant à tous les étages,n'ayant aucune peur du cliché en castagnettes et moralement douteux,"From Paris with love" est fait pour vous.Toutes les productions Besson,via sa filière EuropaCorp ont de quoi faire naître la moquerie.Paris y est encore une fois montrée comme une capitale exotique,mais attention si l'on se dirige vers la banlieue,on y trouve tout plein d'intégristes musulmans prêts à faire sauter le caisson d'une ambassadrice américaine.Nauséeux.Alors on se dit qu'on va au moins pouvoir apprécier la déconnade d'un classique buddy-movie avec le jeune agent aux dents longues obligé de faire équipe avec un vieux lou complètement timbré.Mais d'un côté,on a un Jonathan Rhys-meyers fayot au possible,et de l'autre un John Travolta en total délire narcissico-badass,crâne rasé et bouc proéminent à l'appui.Sinon,il n'y a aucun scénario.Seulement,des poursuites débiles sur l'autoroute,Travolta qui étale 10 mecs à l'arme blanche en 8 mouvements(!!)ou laiss

l'etape suivant est divisé le jeu de données en deux sous-ensembles : pour les données d'entraînement et pour les données de test. Avec train_test_split() , vous n'avez pas besoin de diviser l'ensemble de données manuellement. Par défaut, la fonction train_test_split de Sklearn effectue des partitions aléatoires pour les deux sous-ensembles.

70% pour train et 30% pour test


In [ ]:
from sklearn.model_selection import train_test_split

split_test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=split_test_size, random_state=42) 
                            # test_size = 0.3 is 30%, 42 is the answer to everything

LSTM :

En passe a la derniere etape qui consiste a cration le model d'analyse sentiment on uilise le model LSTM pour notre projet , LTSM est un réseau Long short-term memory (LSTM), en français réseau récurrent à mémoire court et long terme ou plus explicitement réseau de neurones récurrents à mémoire court-terme et long terme,

premièrement en instance le model on a deux sortie soit postive ou negative et adam comme optimazers :

In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X_train.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(optimizer = 'adam',loss = 'binary_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 297, 128)          65536     
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 297, 128)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 2)                 394       
Total params: 320,730
Trainable params: 320,730
Non-trainable params: 0
_________________________________________________________________
None


puis en entraine le model par donnant le nombre d'epechqe 20 qui le nombre de fois pour entraine et batch size eqal 1000

In [ ]:
model.fit(X_train,y_train, epochs = 5, batch_size=1000)


Epoch 1/5
112/112 [==============================] - 159s 1s/step - loss: 0.5222 - accuracy: 0.7470
Epoch 2/5
112/112 [==============================] - 154s 1s/step - loss: 0.3535 - accuracy: 0.8467
Epoch 3/5
112/112 [==============================] - 154s 1s/step - loss: 0.3346 - accuracy: 0.8535
Epoch 4/5
112/112 [==============================] - 153s 1s/step - loss: 0.3292 - accuracy: 0.8569
Epoch 5/5
112/112 [==============================] - 153s 1s/step - loss: 0.3128 - accuracy: 0.8642


En a la precision final equal 0.8642 En comparant cette resultas avec les tranformers en trouve que CamemBERT a une precision eqal = 94.10 dont plus grand donc on focalusé dans la deuxieme section a utiliser les tranfomres pre entriné pour creer un application web qui est cabale de detecter si un commantaire est positive ou negative 

### Utiliser un tranformers

CamemBERT est une “version” de RoBERTa pré-entraînée sur un jeu de données francophone. RoBERTa lui-même est une version de BERT pour laquelle, certains hyperparamètres du pré-entraînement ont été modifiés et l’objectif de prédiction de phrase suivante (Next-Sentence Prediction) a été supprimé. CamemBERT hérite donc des avantages de BERT.

Pour l’implémentation de notre modèle d’analyse de sentiments, nous utiliserons la bibliothèque Transformers de HuggingFace ansi que PyTorch.

In [ ]:
!pip install flask-ngrok
!pip install flask==0.12.2  
!pip install transformers>=4.0
!pip install sentencepiece

on uziper le fhicher projet.rar qui contient lesp pages web et ses codes .css 

In [ ]:
!unrar x projet.rar


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from projet.rar

Creating    static                                                    OK
Creating    static/assets                                             OK
Creating    static/assets/css                                         OK
Extracting  static/assets/css/fontawesome.css                              0%  OK 
Extracting  static/assets/css/owl.css                                      0%  OK 
Extracting  static/assets/css/templatemo-finance-business.css              0%  OK 
Creating    static/assets/fonts                                       OK
Extracting  static/assets/fonts/Flaticon.woff                              0%  OK 
Extracting  static/assets/fonts/flexslider-icon.eot                        0%  OK 
Extracting  static/assets/fonts/flexslider-icon.svg                        1%  OK 
Extracting  static/assets/fonts/flexslider-icon.ttf        

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline


tokenizer = AutoTokenizer.from_pretrained("tblard/tf-allocine")
model = TFAutoModelForSequenceClassification.from_pretrained("tblard/tf-allocine")

nlp = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

All model checkpoint layers were used when initializing TFCamembertForSequenceClassification.

All the layers of TFCamembertForSequenceClassification were initialized from the model checkpoint at tblard/tf-allocine.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertForSequenceClassification for predictions without further training.


Pour web application et pour aller vite, Flask est un framework de développement web en Python. Il en existe d’autres, le plus connu d’entre eux est Django mais avec Flask, on peut déjà rapidement obtenir des résultats, sans trop se perdre en Ce code commence par importer le module Flask 

In [ ]:
from flask import Flask, request, render_template
from flask_ngrok import run_with_ngrok

On donne ensuite un nom à l’application ici ce sera app 

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

Ensuite vient la partie cruciale : définir une page (ou route) avec flask @app.route permet de préciser à quelle adresse ce qui suit va s’appliquer.
Ici comme on est sur la page d’accueil, on met juste (“/”)








Ensuite vient la page index.html qui va s’éxécuter sur la page “/”. 


In [ ]:
@app.route('/')

def my_form():
    return render_template('index.html')

    
@app.route('/', methods=['POST'])
def my_form_post():
    text1 = request.form['message'].lower()

    a = nlp(text1)
    text = a[0]['label']
    score = a[0]['score']
    score = round(score,6)
    if text =='NEGATIVE' :
      coleur = 'red'
    else :
      coleur = 'green'
        
    return render_template('test.html',scor=score,final=text,coleur=coleur) 

if __name__ == "__main__":
    app.run()


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://72f8-104-199-74-189.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [01/Sep/2021 12:00:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 12:00:28] "GET /static/vendor/bootstrap/css/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 12:00:28] "GET /static/assets/css/fontawesome.css HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 12:00:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 12:00:28] "GET /static/assets/css/templatemo-finance-business.css HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 12:00:28] "GET /static/assets/css/owl.css HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 12:00:29] "GET /static/assets/fonts/fontawesome-webfont.woff2?v=4.3.0 HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 12:00:29] "GET /static/vendor/jquery/jquery.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 12:00:29] "GET /static/vendor/bootstrap/js/bootstrap.bundle.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 12:00:29] "GET /static/assets/css/owl.css HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2021 12:00:29] "GET /static/vendor/jquery/jquery.min.js HTTP/1.1